# Introduction

This notebook will go through the basic steps of preparing the input and calculating non-local quantities.

# Input

The first step will be the preparation of input data. For this tutorial we assume, that you have been provided with a file containing the DMFT output data. 

We will convert this file to an input file for the `LadderDGA.jl` code and generate a frequency mesh. 

You can also peak into HDF5 files using the Julia HDF5 package.

In [1]:
using HDF5
path_to_input = "/home/julian/Hamburg/lDGA_checks/triangular_test/U2_n1_b12.h5"
f = h5open(path_to_input)

LoadError: unable to determine if /home/julian/Hamburg/lDGA_checks/triangular_test/U2_n1_b12.h5 is accessible in the HDF5 format (file may not exist)

## TRIQS Conversion

If you are starting out with DMFT data from TRIQS, the `triqs_conv.jl` script in the scripts subdirectory of the `LadderDGA.jl` code will responsible for the conversion. This script also utilizes the `SparseVertex` and `EquivalencyClassesConstructor.jl` packages. 

Most of the conversion scripts are built to be called from the command line with the input given as parameters.
In Julia call parameters are stored in the `ARGS` array. Thus, in order to emulate a call with parameters in jupyter notebooks, we will modify this array befor calls to scripts.

We start by adding the path to a triqs output file and a directory for the script output (`@__DIR__` points to the directory the current script runs in) to the `ARGS` array and executing the `triqs_conv.jl` script for this input.
The input file is NOT provided with this example!

If you want to know more about a function, you can type `? function_name` in any cell to view the docstring.

In [2]:
path_to_input = "/home/julian/Hamburg/lDGA_checks/square_test/U2_n1_b12.h5"
path_to_output = "/home/julian/Hamburg/lDGA_checks/square_test"
empty!(ARGS)
push!(ARGS,path_to_input)
push!(ARGS,path_to_output)
include("../scripts/triqs_conv.jl")

Generating freqency mesh with bosonic -59:59, fermionic -60:59 indices
Constructing Array
Starting Computation 3
  1.792241 seconds (142.60 k allocations: 110.212 MiB, 2.93% gc time, 22.57% compilation time)
  0.129552 seconds (54 allocations: 16.165 MiB)
  0.601458 seconds (11 allocations: 78.148 MiB)


## Config File
You will find two new files in the `path_to_output` directory. Before we can start the calculation we need to specify a configuration file which will point `LadderDGA.jl` to these files and provide all necessary parameters.

You will find an example configuration named `config.toml` in the root directory of `LadderDGA.jl`.
`U`, `mu`, `beta`, `nden` and `kGrid` should be set according to your DMFT calculation. The syntax for the `kGrid` parameter is as follows: The grid is given as a string starting with the grid name (see also `Dispersions.jl` for more information), followed by additional parameters, separated by `-`. Currently `2Dsc` and `3Dsc` are available with a single parameter for the hopping to neighbouring sites. Examples are `"2Dsc-0.25"` or `"3Dsc-0.408248"`

Remember to set the `inputDir` and `freqFile` variables to the previously generated files.

# Running the code

The `LadderDGA.jl` code consists of a collection of functions but no linear program flow. This means one has to define a script which defines the actual program flow.
Some examples can be found in the root directory of the project. Here we will go through a typical program flow, also giving us the opportunity to plot quantities at differnts steps of the calculation.
First, point the `cfg_file` variable to your edited configuration file.
We then also tell Julia to use `Plots` and `LadderDGA`.

LadderDGA.jl can be run in parallel by specifying the `path_to_source` variable. If you want to run the single core version, comment out lines `4` to `10` and uncomment line `13`.

In [3]:
cfg_file = "/home/julian/Hamburg/lDGA_checks/square_test/config.toml"
cfg_file = "/home/julian/Hamburg/lDGA_shift_tests/configs/config_50_100_u1_b12.toml"
path_to_source = "/home/julian/Hamburg/LadderDGA.jl"

using Pkg
Pkg.activate(path_to_source; io=devnull)
using LadderDGA
using Plots

┌ Info: Precompiling LadderDGA [78e40beb-bf89-4c0e-9d2b-bee278912f2b]
└ @ Base loading.jl:1423


The next step will read the config and input files and set up all variables for you.
The `kGrids` (reduced k grid) arrays contain k grids of the sizes specified in the `Nk` array in your `config.toml`. 
This can be handy for finite size scaling, but for now we will stick with simple calculations and only use a single k grid size.
Finally we will also need to read in the DMFT quantities. `setup_LDGA` will do just that.

Note: the full grids (here stored in the kGrids array) are deprecated. They will be removed from future versions, together with most debug variables (all variables in the return list of `readConfig` after `qGridLoc`).

In [4]:
mP, sP, env, kGridStr = readConfig(cfg_file)
kGStr = kGridStr[1]
impQ_sp, impQ_ch, gImp, qGridLoc, qG, gLoc, GLoc_fft, Σ_loc, FUpDo = setup_LDGA(kGStr, mP, sP, env);

[ Info: Reading Inputs...
[ Warning: Could not find E_kin_DMFT, E_pot_DMFT key in input
[ Warning: reading Hubbard Parameters from config. These should be supplied through the input jld2!
[ Info: Setting up calculation for kGrid 3Dsc-0.2041241452319315 of size 4
[ Info: setting usable ranges of sp and ch channel from 37:65 and 37:65 to the same range of 37:65
[ Warning: TODO: update local omega sum with correction
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
┌ Info: Inputs Read. Starting Computation.
│ Local susceptibilities with ranges are:
│ χLoc_sp(37:65) = 0.3182, χLoc_ch(37:65) = 0.1222
└ sum χupup check (fit, tail sub, tail sub + fit, expected): 0.22019288623927485 ?≈? 0.22019288623927483 ?=? 0.22019288623927483 ?≈? 0.25"


In [5]:
v1 = reshape(view(gImp, 1, :), LadderDGA.gridshape(qGridLoc))dd
v2 = reshape(view(gImp, 12, :), LadderDGA.gridshape(qGridLoc))
t = [0.0 + 0.0im, 0.0 + 0.0im]
t2 = LadderDGA.conv_fft(qGridLoc,v1, v2)
LadderDGA.conv_fft!(qGridLoc, t[1:2] ,v1, v2)

LoadError: BoundsError: attempt to access 1×1×1×251 Array{ComplexF64, 4} at index [1, 1:1]

We are now in the position to calculate the ladder DGA self energy. The first 5 lines compute all quantities on a lattice of size 1. The resulting self energy is used to cancel out discretization errors and finite size effects in the full self energy.

In [19]:
bubbleLoc = calc_bubble(gImp, qGridLoc, mP, sP)
locQ_sp = calc_χ_trilex(impQ_sp.Γ, bubbleLoc, qGridLoc, mP.U, mP, sP);
locQ_ch = calc_χ_trilex(impQ_ch.Γ, bubbleLoc, qGridLoc, -mP.U, mP, sP);
Σ_ladderLoc = calc_Σ(locQ_sp, locQ_ch, bubbleLoc, gImp, FUpDo, qGridLoc, mP, sP)
Σ_ladderLoc = Σ_ladderLoc .+ mP.n * mP.U/2.0;


bubble = calc_bubble(GLoc_fft, qG, mP, sP);
nlQ_sp = calc_χ_trilex(impQ_sp.Γ, bubble, qG, mP.U, mP, sP);
nlQ_ch = calc_χ_trilex(impQ_ch.Γ, bubble, qG, -mP.U, mP, sP);
nlQ_sp_nλ = deepcopy(nlQ_sp)
nlQ_ch_nλ = deepcopy(nlQ_ch)
#λ_correction!(:sp, impQ_sp, impQ_ch, FUpDo, Σ_loc, Σ_ladderLoc, nlQ_sp, nlQ_ch, bubble, GLoc_fft, qG, mP, sP)

Σ_ladder = calc_Σ(nlQ_sp, nlQ_ch, bubble, GLoc_fft, FUpDo, qG, mP, sP)
Σ_ladder_corrected = Σ_ladder .- Σ_ladderLoc .+ Σ_loc[1:size(Σ_ladder,1)];

LoadError: MethodError: no method matching *(::SubArray{ComplexF64, 0, Array{ComplexF64, 3}, Tuple{Int64, Int64, Int64}, true}, ::Vector{ComplexF64})
[0mClosest candidates are:
[0m  *(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:655
[0m  *([91m::Union{SparseArrays.AbstractSparseMatrixCSC{TA, Ti}, SubArray{TA, 2, <:SparseArrays.AbstractSparseMatrixCSC{TA, Ti}, Tuple{Base.Slice{Base.OneTo{Int64}}, I}} where I<:AbstractUnitRange} where Ti[39m, ::Union{StridedVector, BitVector}) where TA at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.7/SparseArrays/src/linalg.jl:50
[0m  *([91m::FFTW.cFFTWPlan{T, K, true}[39m, ::StridedArray{T}) where {T, K} at /home/julian/.julia/packages/FFTW/kKdEk/src/fft.jl:725
[0m  ...

# Visualization

The main advantage of calculations in jupyter notebooks are the convenient options for data visualization. The following cells give examples for plotting in Julia

In [9]:
LadderDGA.sum_freq_full!(t2, sP.sh_f, 1.0, sP.fνmax_cache_r, sP.fνmax_lo, sP.fνmax_up)

LoadError: UndefVarError: t2 not defined

In [10]:
sh = LadderDGA.get_sum_helper(nlQ_ch.usable_ω, sP, :b)
χch_ω = LadderDGA.kintegrate(qG, nlQ_ch.χ, 1)[1,:]
χch_sum = real(LadderDGA.sum_freq(χch_ω[nlQ_ch.usable_ω], [1], sh, mP.β)[1])
real(impQ_ch.χ_loc + impQ_sp.χ_loc - χch_sum)

sh = LadderDGA.get_sum_helper(nlQ_sp.usable_ω, sP, :b)
χsp_ω = LadderDGA.kintegrate(qG, nlQ_sp.χ, 1)[1,:]
χsp_sum = real(LadderDGA.sum_freq(χsp_ω[nlQ_sp.usable_ω], [1], sh, mP.β)[1])

LoadError: UndefVarError: nlQ_ch not defined

In [11]:
χchED_ω_naive = sum(impQ_ch.χ,dims=[1,2])[1,1,:]/(mP.β^2);
χspED_ω_naive = sum(impQ_sp.χ,dims=[1,2])[1,1,:]/(mP.β^2);

reduction_prct = -0.1
χchED_ω_fit = [LadderDGA.sum_freq(impQ_ch.χ[:,:,i], [1,2], sP.sh_f, mP.β)[1,1] for i in 1:size(impQ_ch.χ,3)];
χspED_ω_fit = [LadderDGA.sum_freq(impQ_sp.χ[:,:,i], [1,2], sP.sh_f, mP.β)[1,1] for i in 1:size(impQ_sp.χ,3)];
usable_ch = LadderDGA.find_usable_interval(real(χchED_ω_fit), reduce_range_prct=reduction_prct)
usable_sp = LadderDGA.find_usable_interval(real(χspED_ω_fit), reduce_range_prct=reduction_prct)
usable_fit_DMFT = intersect(usable_ch, usable_sp)

usable_naive_ch = LadderDGA.find_usable_interval(real(χchED_ω_naive), reduce_range_prct=reduction_prct)
usable_naive_sp = LadderDGA.find_usable_interval(real(χspED_ω_naive), reduce_range_prct=reduction_prct)


plot(usable_ch .- sP.n_iω .- 1, real(χchED_ω_fit[usable_ch]), markershape=:auto, label="richardson", c=:green)
plot!(usable_ch .- sP.n_iω .- 1, repeat([0], length(usable_ch)), fillrange=real(χchED_ω_fit[usable_ch]), fillalpha=0.06, c=:green, label=nothing)

plot!(usable_naive_ch .- sP.n_iω .- 1, real(χchED_ω_naive[usable_naive_ch]), markershape=:auto, label="naive", xlabel="ωₙ", ylabel="χ_ch", c=:orange, title="reduction: $(reduction_prct*100)%")
plot!(usable_naive_ch .- sP.n_iω .- 1, repeat([0], length(usable_naive_ch)), fillrange=real(χchED_ω_naive[usable_naive_ch]), fillalpha=0.3, c=:orange, label=nothing)

plot!(usable_naive_ch .- sP.n_iω .- 1, real(χch_ω[usable_naive_ch]), markershape=:auto, label="Σ_q χ_ch^lDGA", xlabel="ωₙ", ylabel="χ_ch", c=:orange, title="reduction: $(reduction_prct*100)%")

[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: u

[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
[ Warning: u

LoadError: UndefVarError: χch_ω not defined

In [12]:
plot(imag.(Σ_loc[1:6]), markershape=:auto, label="Σ DMFT")
plot!(imag.(Σ_ladderLoc[1:6]), markershape=:auto, label="Σ_loc lDGA")

LoadError: UndefVarError: Σ_ladderLoc not defined

In [13]:
Σ_ladderLoc

LoadError: UndefVarError: Σ_ladderLoc not defined

In [14]:
plot(real.(impQ_sp.χ_ω),markershape=:auto, label="χ^sp_ω DMFT")
plot!(real.(χsp_ω),markershape=:auto, label="Σ_q χ^sp_ω lDGA")

LoadError: UndefVarError: χsp_ω not defined

In [15]:
plot(real.(impQ_ch.χ_ω),markershape=:auto, label="χ^ch_ω DMFT")
plot!(real.(χch_ω),markershape=:auto, label="Σ_q χ^ch_ω lDGA")

LoadError: UndefVarError: χch_ω not defined

In [16]:
corr = LadderDGA.Σ_correction(1:size(bubble,3), bubble, FUpDo, sP);
corr_loc = LadderDGA.Σ_correction(1:size(bubbleLoc,3), bubbleLoc, FUpDo, sP);

LoadError: UndefVarError: bubble not defined

In [17]:
ν_axis = LadderDGA.ν_axis
q_axis = LadderDGA.q_axis
ω_axis = LadderDGA.ω_axis

3

In [18]:
using LaTeXStrings
corrLoc = LadderDGA.Σ_correction(1:size(bubbleLoc,3), bubbleLoc, FUpDo, sP);
corr = LadderDGA.Σ_correction(1:size(bubbleLoc,3), bubble, FUpDo, sP);
t = [-sP.n_iν:sP.n_iν-1,-sP.n_iω:sP.n_iω]
hms = [heatmap(transpose(real(corr[:,1,:])), ylabel=L"\omega_n", xlabel=L"\nu_n",title=L"\mathrm{tmp}_\mathrm{loc}(\omega,\nu)")
    heatmap(transpose(real(corr_loc[:,1,:])),ylabel=L"\omega_n", xlabel=L"\nu_n", title=L"\mathrm{tmp}_\mathrm{loc, shift}(\omega,\nu)")]
plot(hms..., layout=(1,2), size=(800,300))

LoadError: UndefVarError: bubble not defined